# Transformación de los datos en bruto

## Modulos a emplear

In [1]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
import sys
import os

# Funciones importadas como modulos
notebook_dir = os.getcwd()
scripts_dir = os.path.join(notebook_dir, "..", "src")
sys.path.append(scripts_dir)
from data_transform.load_csv import load_csv
from data_transform.map_cimas_to_depth_multiple import map_cimas_create_columns

# Codigo para imprimir mas de una salida de la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Cargamos la informacion de los registros de pozo y cimas

In [2]:
# Definimos las rutas de los registros de pozo y cimas 
logs_file_name = "logs_pijije_rhob_nphi_dtco_syn.csv"
tops_file_name = "tops_pijije_rhob_nphi_dtco_syn.csv"
path_well_logs = os.path.join(notebook_dir, "..", "data", "raw", logs_file_name)
path_well_tops = os.path.join(notebook_dir, "..", "data", "raw", tops_file_name)

# Cargamos la informacion de los pozos
well_logs_df = load_csv(path_well_logs)

# Cambiamos el tipo de de columnas a float
columnas_convertir = ["md", "dtco","dtsm", "gr", "rp"]
well_logs_df[columnas_convertir] = well_logs_df[columnas_convertir].astype(float)

# Verificamos que no exista ninguna valor negativo en md
well_logs_df = well_logs_df[well_logs_df["md"] >= 0]

# Cargamos la informacion de las cimas
well_tops_df = load_csv(path_well_tops)

# Cambiamos el tipo de columna a float
well_tops_df["md"] = well_tops_df["md"].astype(float)

# Ordena el DataFrame de cimas por 'MD' para asegurarte de que esté en el orden correcto
well_tops_df = well_tops_df.sort_values(by='md')

# Borramos la cima "PT"
well_tops_df = well_tops_df[well_tops_df["surface"] != "PT"]

## Juntamos las cimas con los pozos

In [3]:
# Mapeamos las cimas con los registros de pozo 
pijije_df = map_cimas_create_columns(well_tops_df, well_logs_df, False)

## Definimos el campo al que pertenecen los pozos

In [4]:
# Definimos el campo al que pertenecen los pozos
field = "Caparroso-Pijije-Escuintle"
pijije_df["field"] = field

## Creamos el archivo SQL para el llenado de la tabla wells

In [6]:
# Agrupamos por wellname y calculamos el mínimo y máximo de md, y el campo field
grouped_df = pijije_df.groupby('wellname').agg({'md': ['min', 'max'], 'field': 'first'}).reset_index()

# Renombramos las columnas para mayor claridad
grouped_df.columns = ['wellname', 'min_md', 'max_md', 'field']

         wellname      min_md       max_md                       field
0    Caparroso-71  499.903015  6120.719727  Caparroso-Pijije-Escuintle
1    Caparroso-75   60.000000  5614.065600  Caparroso-Pijije-Escuintle
2      Pijije-101    0.000000  5233.111200  Caparroso-Pijije-Escuintle
3      Pijije-111    0.000800  5851.856000  Caparroso-Pijije-Escuintle
4      Pijije-112    0.062000  5149.962891  Caparroso-Pijije-Escuintle
5      Pijije-117    0.000000  5636.056641  Caparroso-Pijije-Escuintle
6      Pijije-125   40.081200  5271.058800  Caparroso-Pijije-Escuintle
7      Pijije-127   46.024799  5414.924316  Caparroso-Pijije-Escuintle
8       Pijije-1A    0.000000  6030.925200  Caparroso-Pijije-Escuintle
9       Pijije-21    0.000000  5332.018800  Caparroso-Pijije-Escuintle
10      Pijije-25    0.076400  5900.547200  Caparroso-Pijije-Escuintle
11  Pijije-25REE2    0.000000  6049.975200  Caparroso-Pijije-Escuintle
12      Pijije-27   27.600000  5379.900000  Caparroso-Pijije-Escuintle
13   P

In [7]:
# Crear una lista para almacenar las consultas SQL
sql_queries = []

# Iterar sobre las filas del DataFrame
for index, row in grouped_df.iterrows():
    wellname = row['wellname']
    min_md = row['min_md']
    max_md = row['max_md']
    field = row['field']

    # Crear la cadena SQL para la entrada
    sql_query = f"INSERT INTO wells (well, field, top_md, bottom_md) VALUES ('{wellname}', '{field}', {max_md}, {min_md});"
    
    # Agregar la consulta SQL a la lista
    sql_queries.append(sql_query)

# Guardar las consultas SQL en un archivo .sql
sql_file_name = "insert_queries.sql"
path = os.path.join(notebook_dir, "..", "sql", sql_file_name)

with open(path, "w") as sql_file:
    for query in sql_queries:
        sql_file.write(query + "\n")

print(f"Archivo {sql_file_name} creado exitosamente en {path}.")

141

123

120

122

140

...

122

119

121

119

Archivo insert_queries.sql creado exitosamente en d:\Usuarios\574905\Documents\Python\CPE\Sinteticos\CPE\notebooks\..\sql\insert_queries.sql.


## Creamos el archivo SQL para el llenado de la tabla logs

## Creamos el DataSet con la informacion procesada

In [9]:
# Guardamos el DataSet pijije_procesed en formato csv
processed_file_name = "data_pijije_rhob_nphi_dtco_syn.csv"
path_to_save = os.path.join(notebook_dir, "..", "data", "processed", processed_file_name)
pijije_df.to_csv(path_to_save, index=False)